In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
from src.rulm import *
from datasets import load_dataset

C:\ProgramData\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = RuLMDataset(base_path="data/rulm")

In [4]:
dataset.download_and_prepare()

In [5]:
train_data = dataset.as_dataset(split='train[:10%]')

In [6]:
train_data

Dataset({
    features: ['text', 'meta'],
    num_rows: 1481103
})

In [9]:
train_data = data["train"]

KeyError: "Column train not in the dataset. Current columns in the dataset: ['text', 'meta']"

In [7]:
train_data = train_data.remove_columns([                                                             
        col for col in train_data.column_names if col != "text"                                        
    ])

In [8]:
print (train_data[ 1000 ])

{'text': 'Вооруженные силы Таиланда в ходе штурма укреплений оппозиционеров в центре Бангкока прорвали баррикады "краснорубашечников", сообщает Reuters. По последним данным, в ходе столкновений сторонников бывшего премьера Таксина Чинавата с военными погибли пять человек, в том числе репортер из Италии. По данным официальных властей Таиланда, которые приводит AFP, лагерь оппозиции покинули практически все идеологи восстания. Тем не менее, столкновения властей с протестантами продолжаются. Число пострадавших составляет десять человек, как минимум двое из них - иностранные репортеры. Точных сведений о личности погибших, за исключением информации о скончавшемся по пути в больницу журналисте, не поступало. Предположительно, двое из них являются "краснорубашечниками". Источник AFP в полиции рассказал, что военным поступил приказ вести огонь по протестующим. Штурм укреплений "краснорубашечников" начался утром в среду, 19 мая. Днем ранее правительство Таиланда отказалось от предложения оппози

In [9]:
def batch_iterator(dataset, batch_size=1_000):                                                       
    for batch in dataset.iter(batch_size=batch_size):                                                
        yield batch["text"]

In [10]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("models/qwen/", local_files_only=True)

In [11]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_numbers',
 '(a',
 ',',
 'Ġb',
 '):Ċ',
 'ĠĠĠ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`',
 '."""Ċ',
 'ĠĠĠ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

In [12]:
tokenizer = old_tokenizer.train_new_from_iterator(                                        
        batch_iterator(train_data),                                                                    
        vocab_size=32_768,                                                                       
    ) 

In [13]:
tokenizer.save_pretrained("qwen_32768")

('qwen_32768\\tokenizer_config.json',
 'qwen_32768\\special_tokens_map.json',
 'qwen_32768\\vocab.json',
 'qwen_32768\\merges.txt',
 'qwen_32768\\added_tokens.json',
 'qwen_32768\\tokenizer.json')

In [14]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_n',
 'umb',
 'ers',
 '(a',
 ',',
 'Ġb',
 '):Ċ',
 'ĠĠĠ',
 'Ġ"',
 '""',
 'Add',
 'Ġthe',
 'Ġt',
 'wo',
 'Ġn',
 'umb',
 'ers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`',
 '."',
 '"',
 '"Ċ',
 'ĠĠĠ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']